In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [4]:
pipeline.preprocess()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:01<00:00, 965.84it/s]


In [5]:
pipeline.create_tfidf_embeddings()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 9839.75it/s]


In [6]:
pipeline.create_tfidf_feature()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 684.78it/s]


In [7]:
pipeline.create_jaccard_feature()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:01<00:00, 1495.98it/s]


In [8]:
pipeline.create_sentence_features()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 60650.48it/s]


In [9]:
pipeline.create_interpretation_features()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 1760.15it/s]


In [10]:
pipeline.create_POS_features()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:01<00:00, 1120.27it/s]


In [11]:
pipeline.features

,qID,pID,tfidf_cosine,tfidf_euclidean,tfidf_manhattan,jaccard,words_doc,words_query,words_difference,words_rel_difference,...,subjectivity_doc,polarity_doc,subjectivity_query,polarity_query,doc_nouns,doc_adjectives,doc_verbs,query_nouns,query_adjectives,query_verbs
0,651052,121251,0.852590,0.542973,1.896362,0.200000,38,6,32.0,6.333333,...,0.500000,0.050000,0.687500,-0.3125,10,3,7,2,1,1
1,485808,136865,0.481687,1.018148,5.600224,0.102564,107,6,101.0,17.833333,...,0.594444,0.178704,0.666667,-0.7000,32,11,15,2,2,1
2,550996,149222,0.455940,1.043130,4.301138,0.130435,58,11,47.0,5.272727,...,0.217143,0.025714,0.000000,0.0000,11,4,8,2,0,4
3,632071,154108,0.000000,1.000000,6.205483,0.000000,83,5,78.0,16.600000,...,0.448864,0.056818,0.000000,0.0000,27,13,10,1,0,2
4,726432,159582,0.562177,0.935760,4.339546,0.120000,61,6,55.0,10.166667,...,0.625000,-0.375000,0.000000,0.0000,18,3,13,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,595630,8570301,0.260758,1.215929,4.647269,0.095238,60,6,54.0,10.000000,...,0.436364,0.090909,0.000000,0.0000,24,2,6,2,1,2
1496,678422,881900,0.634672,0.854784,3.813911,0.041667,54,4,50.0,13.500000,...,0.220833,-0.145833,0.000000,0.0000,13,3,8,1,0,1
1497,1164923,3705384,0.386200,1.107971,5.632179,0.093750,66,5,61.0,13.200000,...,0.455000,0.135000,0.000000,0.0000,23,7,7,3,0,0
1498,662044,1740760,0.458970,1.040221,4.665828,0.086957,39,6,33.0,6.500000,...,0.383333,-0.016667,0.000000,0.0000,10,7,6,1,0,2
